In [2]:
import implicit
import lightfm
import pandas as pd 
import numpy as np
from implicit.bpr import BayesianPersonalizedRanking as BPR
from scipy import sparse
import orion_recommend
from orion_recommend.evaluate import metrics
import lightfm

In [2]:
users = pd.read_csv("users", index_col = 0)
items = pd.read_csv("items", index_col = 0)
interactions = pd.read_csv("interactions", index_col = 0)

In [3]:
train = np.load("new_fa_train_data.npy")
test = np.load("new_fa_test_data.npy")

In [4]:
ui_matrix = np.load("fa_ui_matrix.npy")

In [5]:
ua_matrix = np.load("new_fa_ua_matrix.npy")

In [6]:
ua_matrix.shape

(86317, 471)

### Using user-item matrix


In [7]:
ui_matrix.shape, train.shape, test.shape

((86317, 10392), (76163, 15), (19041, 15))

In [8]:
test_users = np.unique(test[:,0])
train_users = np.unique(train[:,0])

In [9]:
test_items = np.unique(test[:,3])
train_items = np.unique(train[:,3])

In [10]:
test_users.shape, train_users.shape

((18645,), (70196,))

In [11]:
test_items.shape, train_items.shape

((6452,), (9989,))

In [12]:
intersection_users = np.intersect1d(train_users,test_users)
intersection_items =  np.intersect1d(train_items,test_items)

In [13]:
ind_users = pd.Series(test_users).isin(intersection_users)
ind_items = pd.Series(test_items).isin(intersection_items)

In [14]:
test_users = test_users[~ind_users]
test_items = test_items[~ind_items]

In [15]:
test_users.shape[0] + train_users.shape[0] - ui_matrix.shape[0] == 0

True

In [16]:
test_items.shape[0] + train_items.shape[0] - ui_matrix.shape[1] == 0

True

In [17]:
ui_train = ui_matrix[train_users]
ui_test = ui_matrix[test_users]

In [18]:
iu_train = ui_matrix.T[train_items]
iu_test = ui_matrix.T[test_items]

In [19]:
ui_train.shape, ui_test.shape

((70196, 10392), (16121, 10392))

In [20]:
train_sparse = sparse.coo_matrix(ui_train)
test_sparse = sparse.coo_matrix(ui_test)

In [21]:
test_sparse.shape

(16121, 10392)

In [22]:
train_sparse.shape

(70196, 10392)

### User-Attribute

In [23]:
ua_matrix.shape, train.shape, test.shape

((86317, 471), (76163, 15), (19041, 15))

In [24]:
test_users = np.unique(test[:,0])
train_users = np.unique(train[:,0])

In [25]:
test_items = np.unique(test[:,3])
train_items = np.unique(train[:,3])

In [26]:
test_users.shape, train_users.shape

((18645,), (70196,))

In [27]:
test_items.shape, train_items.shape

((6452,), (9989,))

In [28]:
test_users.shape[0] + train_users.shape[0] - ua_matrix.shape[0] == 0

False

In [29]:
ua_train = ua_matrix[train_users]
ua_test = ua_matrix[test_users]

In [30]:
ua_train.shape, ua_test.shape

((70196, 471), (18645, 471))

In [31]:
train_sparse = sparse.coo_matrix(ua_train)
test_sparse = sparse.coo_matrix(ua_test)

In [32]:
test_sparse.shape

(18645, 471)

In [33]:
train_sparse.shape

(70196, 471)

In [34]:
from lightfm import cross_validation

In [35]:
train, test = cross_validation.random_train_test_split(sparse.coo_matrix(ui_matrix), test_percentage=0.2, random_state=None)

In [10]:
user_features = np.load("user_attributes_npc.npy")

In [11]:
user_features.shape

(86297, 60)

In [12]:
user_features = sparse.coo_matrix(user_features)

In [13]:
user_features.shape

(86297, 60)

In [14]:
item_features = np.load("new_fa_ia_matrix.npy")

In [16]:
item_features = sparse.coo_matrix(item_features)

In [17]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k


# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(train, 
          item_features=item_features, epochs=30)

NameError: name 'train' is not defined

In [49]:
# Evaluate the trained model
test_precision = precision_at_k(model, test,train,
          item_features=item_features,  k=10)

In [54]:
test_precision.mean()

0.0002973455

In [52]:
train_precision = precision_at_k(model, train, 
          item_features=item_features,  k=10)

In [53]:
train_precision.mean()

0.035814065

In [60]:
recall_at_k(model,train,
          item_features = item_features,  k=10).mean()

0.2915999087721299